In [2]:
import urllib3

pool = urllib3.HTTPSConnectionPool(
    '13.35.125.81',
    # assert_hostname = "www.okx.com",
    # HTTPSConnection 使用以下选项之一 assert_fingerprint ， assert_hostname 和 host 按此顺序验证连接。如果 assert_hostname 为假，则不进行任何验证。
    server_hostname = "www.okx.com",
)
a  = pool.urlopen(
    "GET",
    "/api/v5/public/time",
    headers={"Host":"www.okx.com"},
    # assert_same_host=False,
    # assert_same_host -- 如果 True 将确保池请求的主机一致，否则将引发HostChangedError。什么时候 False ，您可以在HTTP代理上使用该池并请求外部主机。
)

print(a.status, type(a.status))
print(a.data.decode())

import time
print( time.time())


200 <class 'int'>
{"code":"0","data":[{"ts":"1663151941081"}],"msg":""}
1663151940.44398


In [3]:
import urllib3
url = 'https://www.okx.com/api/v5/public/time'
p = urllib3.util.url.parse_url(url)
print(p.scheme, 
p.port,
p.hostname,
p.path,)
type(p)

https None www.okx.com /api/v5/public/time


urllib3.util.url.Url

In [ ]:
import requests

# header = {"Host": 'www.okx.com'}
# url = 'https://13.35.125.81/api/v5/public/time'
# response = requests.get(url, headers=header, verify=False)
# print(response.json())


class HostHeaderSSLAdapter(requests.adapters.HTTPAdapter):
    def resolve(self, hostname):
        # a dummy DNS resolver
        import random
        ips = [
            # '104.16.89.20',  # CloudFlare
            # '151.101.2.109',  # Fastly
            '16.163.225.163',
        ]
        resolutions = {
            'www.okx.com': random.choice(ips),
        }
        return resolutions.get(hostname)

    def send(self, request, **kwargs):
        from urllib.parse import urlparse

        connection_pool_kwargs = self.poolmanager.connection_pool_kw

        result = urlparse(request.url)
        resolved_ip = self.resolve(result.hostname)

        if result.scheme == 'https' and resolved_ip:
            request.url = request.url.replace(
                'https://' + result.hostname,
                'https://' + resolved_ip,
            )
            connection_pool_kwargs['server_hostname'] = result.hostname  # SNI
            connection_pool_kwargs['assert_hostname'] = result.hostname

            # overwrite the host header
            request.headers['Host'] = result.hostname
        else:
            # theses headers from a previous request may have been left
            connection_pool_kwargs.pop('server_hostname', None)
            connection_pool_kwargs.pop('assert_hostname', None)

        return super(HostHeaderSSLAdapter, self).send(request, **kwargs)



# url = 'https://cdn.jsdelivr.net/npm/bootstrap/LICENSE'
url = 'https://www.okx.com/api/v5/public/time'

session = requests.Session()
session.mount('https://', HostHeaderSSLAdapter())

r = session.get(url)
print(r.headers)




In [ ]:
import requests
from forcediphttpsadapter.adapters import ForcedIPHTTPSAdapter
requests.urllib3.disable_warnings() 

session = requests.Session()
session.mount("https://www.okx.com", ForcedIPHTTPSAdapter(dest_ip='13.35.125.81'))
response = session.get(
    'https://www.okx.com/api/v5/public/time', verify=False)

print( response.json() )

{'code': '0', 'data': [{'ts': '1663053562450'}], 'msg': ''}


In [ ]:
import requests
from requests_toolbelt.adapters.host_header_ssl import HostHeaderSSLAdapter

s = requests.session()
s.mount('https://', HostHeaderSSLAdapter())
res = s.get("https://13.35.125.81/api/v5/public/time", headers={"Host": "www.okx.com"})
print( res.json() )

